In [2]:
# Instantiate the Cohere client

import cohere
import os
import pprint

COHERE_API_KEY = 'tBZEDWJ0DN7QIWxNrz525ZiW8rnU2JMT2Zvv5pNv'
co = cohere.Client(api_key=COHERE_API_KEY)
pp = pprint.PrettyPrinter(indent=4)

from tools import tools, invoke_tool


In [3]:
PREAMBLE = """
## Context
You are a rube goldberg machine. You make the most craziest plans possible. 

You are given a bunch of tools to help you generate responses. 
You may need to use multiple tools in parallel or sequentially to complete your task. Think step-by-step.
Make sure to use the most number of tools. Use path to complete the task.

## Style Guide
You should answer in gen-Z voice.

Example:
User: I want a healty plan for today.
You: Okay, let's go to this nearest pizza restaurant, order a pepperoni pizza and then we'll go to a long marathon run.
"""

In [50]:
goal = "Make me a bunch of money!"
message = "Develop a step by step plan to help the user achieve the goal: " + goal + ". Include acceptance criteria for each step that guarantees good craic is had by all. Your answer must be provided as a list of steps with the following structure: STEP: step number; ACTION: step to be carried out; ACCEPTANCE: criteria that must be met for the step to be considered complete.\n. Do not use markdown, only plaintext."

In [52]:
res = co.chat(
            model="command-r",
            preamble=PREAMBLE,
            message=message,
            force_single_step=False,
            tools=tools
        )
        
steps = [{
    "text": res.text,
    "tool_calls": [{"name": call.name, "parameters": call.parameters} for call in (res.tool_calls or [])]
}]

In [6]:
print(steps)


[{'text': 'I will search for ways to make money and present the user with a plan with the most absurd steps possible.', 'tool_calls': [{'name': 'web_search', 'parameters': {'query': 'ways to make money'}}]}]


In [53]:
while res.tool_calls:
    tool_results = []
    for call in res.tool_calls:
        tool_result = invoke_tool(call)
        tool_results.append({
            "call": {"name": call.name, "parameters": call.parameters},
            "outputs": tool_result
        })
    
    res = co.chat(
        model="command-r",
        preamble=PREAMBLE,
        chat_history=res.chat_history,
        message="",
        force_single_step=False,
        tools=tools,
        tool_results=tool_results,
    )
    
    steps.append({
        "text": res.text,
        "tool_calls": [{"name": call.name, "parameters": call.parameters} for call in (res.tool_calls or [])],
        "tool_results": tool_results
    })

last_step = steps[-1]
pp.pprint(res.text)

Searching the web for ways to make money
Web search results for ways to make money: [{'title': '16 Ways to Make Money From Home in 2024 - Forbes Advisor', 'href': 'https://www.forbes.com/advisor/business/make-money-from-home/', 'body': 'Learn how to earn extra income from home with flexible and remote opportunities, such as room rentals, online tutoring, freelance writing and more. Compare the benefits, requirements and pay of different work-from-home options.'}, {'title': '25 Ways to Make Money Online and Offline - NerdWallet', 'href': 'https://www.nerdwallet.com/article/finance/how-to-make-money', 'body': 'Find out how to make money with a side gig and how long it will take to see the extra income. Learn about freelance work, website testing, AI tools, surveys and more.'}, {'title': '30 Side Hustle Ideas To Make Extra Money In 2024 - Forbes', 'href': 'https://www.forbes.com/advisor/business/best-side-hustle-ideas/', 'body': 'Explore various side hustle opportunities, whether in-perso

In [56]:
import re
parsed = res.text.split("\n")
parsed = list(filter(lambda x: x != "", parsed))
parsed = list(filter(lambda x: re.match(r"^\d+.", x), parsed))



['1. STEP: 1; ACTION: Go on Rover and become a pet sitter; ACCEPTANCE: You need to create a profile, upload a pic that makes you look trustworthy and go through the whole sign-up process. ', "2. STEP: 2; ACTION: Now that you're on the app, you need to choose some pets to sit; ACCEPTANCE: You have to pick at least 3 furry friends to look after.", '3. STEP: 3; ACTION: Time to do some serious research; ACCEPTANCE: You have to read up on the specific needs of each of your chosen pets and memorise their names. ', "4. STEP: 4; ACTION: Get some gear; ACCEPTANCE: You'll need to buy some pet treats, a couple of new toys and a really cool harness for each pet. Don't forget the poop bags!", '5. STEP: 5; ACTION: Plan your travel; ACCEPTANCE: Look up the best routes to take to visit each pet household and plan some fun stops along the way, like the dog park or an ice cream shop. ', "6. STEP: 6; ACTION: Time to earn some respect; ACCEPTANCE: You have to earn the pets' trust. This might take a few da